## Feature Engineering on the processed dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

In [2]:
# Read in the data
GEO = ["texas", "florida"]

if isinstance(GEO, str):
    df = pd.read_csv(f"../../data/processed/{GEO}_auto_processed.csv")
elif isinstance(GEO, list):
    dfs = []
    for geo in GEO:
        df = pd.read_csv(f"../../data/processed/{geo}_auto_processed.csv")
        df["source"] = geo
        dfs.append(df)
    df = pd.concat(dfs)

# eval list columns
df.description = df.description.apply(lambda x: eval(x))
df.comments = df.comments.apply(lambda x: eval(x))
df.amenities = df.amenities.apply(lambda x: eval(x))

df.reset_index(drop=True, inplace=True)

print(df.shape)
df.head()

(1546, 9)


,review_id,listing_id,description,comments,sentiment,label,name,amenities,source
0,83097,5456,"[fabulous, location, walking, convention, cent...","[sylvia, very, nice, informal, relaxed, arrive...",4.333333,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","[TV, Wifi, Air conditioning, Kitchen, Pets liv...",texas
1,133337,5456,"[fabulous, location, walking, convention, cent...","[sylvia, picked, airport, gave, beautiful, eve...",4.000000,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","[TV, Wifi, Air conditioning, Kitchen, Pets liv...",texas
2,150928,5456,"[fabulous, location, walking, convention, cent...","[lovely, time, enjoyed, apartment, clean, spac...",3.000000,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","[TV, Wifi, Air conditioning, Kitchen, Pets liv...",texas
3,2706775,5456,"[fabulous, location, walking, convention, cent...","[sylvia, excellent, host, stayed, touch, made,...",4.500000,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","[TV, Wifi, Air conditioning, Kitchen, Pets liv...",texas
4,8602878,5456,"[fabulous, location, walking, convention, cent...","[place, cute, little, self, contained, cottage...",4.000000,good,"['walk', '6th', 'rainey', 'st', 'convention', ...","[TV, Wifi, Air conditioning, Kitchen, Pets liv...",texas


### Distribution of labels

First, let's look at the distribution of labels in the processed dataset.

In [3]:
mbad_pcnt = df[df.label == "mbad"].shape[0] / df.shape[0]
mgood_pcnt = df[df.label == "mgood"].shape[0] / df.shape[0]
mbad_or_mgood_pcnt = df[df.label.isin(["mbad", "mgood"])].shape[0] / df.shape[0]
print(
    f"mbad: {mbad_pcnt:.2%}, mgood: {mgood_pcnt:.2%}, mbad or mgood: {mbad_or_mgood_pcnt:.2%}"
)

bar_vals = df.label.value_counts()

fig = px.bar(
    bar_vals,
    x=bar_vals.index,
    y=bar_vals.values,
    title=f"Distribution of labels (GEO={GEO})",
    labels={"index": "Label", "y": "Count"}
)

fig.show()

mbad: 8.02%, mgood: 6.86%, mbad or mgood: 14.88%


## Correlation filter helper function
This function allows us to filter down to only include highly correlated features with the label.

In [4]:
def corr_filter(features, corr_thresh=0.05) -> pd.DataFrame:
    """Filter features based on correlation with label."""

    if corr_thresh is None:
        return features

    to_filter = features.copy()
    
    to_filter["review_id"] = df.review_id
    to_filter["label"] = df.label

    # drop na labels. Thanks Charlie :/
    to_filter = to_filter.dropna(subset=["label"])

    # Turn labels into numbers
    labels_list: list = to_filter.label.unique().tolist()
    to_filter.label = to_filter.label.apply(lambda label: labels_list.index(label))

    # calculate correlation of features with label
    features_to_filter = to_filter.drop(["review_id", "label"], axis=1)
    corrs = features_to_filter.corrwith(to_filter.label).sort_values(ascending=False)

    # filter features
    highly_correlated_features = corrs[corrs.abs() > corr_thresh].index
    # drop features with nan correlation
    highly_correlated_features = highly_correlated_features.dropna()

    # filter features
    print(f"Features above correlation threshold ({corr_thresh}):")
    print(highly_correlated_features.tolist())
    filtered = features[highly_correlated_features]
    
    return filtered

### Ngrams

What are the most common ngrams for the different types of reviews? What are the most common overlaps?

In [5]:
def get_consecutive_ngrams(review, n) -> list:
    """Helper function to get ngrams from a review.

    Args:
        review (str): The review to get ngrams from.
        n (int): The number of ngrams to get.

    Returns:
        list: the list of ngrams, joined by underscores.
    """
    if isinstance(review, str):
        review = review.split("")

    return ["_".join(review[i:i+n]) for i in range(len(review)-n-1)]

def series_to_ngrams(series: pd.Series, N):
    n_grams = series.apply(lambda x: get_consecutive_ngrams(x, N))
    return n_grams.explode()

def get_exclusive_sizes(good: set, bad: set, mgood: set, mbad: set) -> dict:
    """Get the sizes of the exclusive sets"""

    only_good = good - bad - mgood - mbad
    only_bad = bad - good - mgood - mbad
    only_mgood = mgood - good - bad - mbad
    only_mbad = mbad - good - bad - mgood
    
    return {
        "good": len(only_good),
        "bad": len(only_bad),
        "mgood": len(only_mgood),
        "mbad": len(only_mbad)
    }

We can look at the number of unique values to each label for different values of N on a line chart.

In [6]:
import plotly.express as px

intersection_sizes = []
for n in [1, 2, 3, 4, 5, 6]:

    good_ngrams = series_to_ngrams(df[df.label == "good"].comments, n).value_counts()
    bad_ngrams = series_to_ngrams(df[df.label == "bad"].comments, n).value_counts()
    mgood_ngrams = series_to_ngrams(df[df.label == "mgood"].comments, n).value_counts()
    mbad_ngrams = series_to_ngrams(df[df.label == "mbad"].comments, n).value_counts()
    intersection_sizes_n = get_exclusive_sizes(
        set(good_ngrams.index),
        set(bad_ngrams.index),
        set(mgood_ngrams.index),
        set(mbad_ngrams.index)
    )

    intersection_sizes_n["n"] = n

    intersection_sizes.append(intersection_sizes_n)


# prepare data
is_df = pd.DataFrame(intersection_sizes)
is_df = is_df.melt(id_vars="n", var_name="label", value_name="count")
is_df["count"] = is_df["count"].astype(int)
is_df["n"] = is_df["n"].astype(int)

# exclude good 
is_df = is_df[is_df.label != "good"]

# plot a line chart
px.line(is_df, 
    x="n", y="count",
    color="label",
    title=f"Unique # of {GEO} n-grams for each label", 
    labels={"n": "N-gram size", "count": "Unique # of n-grams"},
    markers=True
    )

We can see that the number of unique ngrams increases until about n=3, and then levels off. What are the top 10 3-grams for misleading reviews?

In [7]:
import numpy as np
from collections import Counter

# get subset of misleading reviews
subset = df[df.label.isin(["mbad", "mgood"])]

# get groups of N consecutive words
N = 3
n_grams = series_to_ngrams(subset.comments, N).to_list()

print(f"Top 10 most common {N}-grams in misleading reviews:")
Counter(n_grams).most_common(10)

Top 10 most common 3-grams in misleading reviews:


[('within_walking_distance', 11),
 ('would_not_recommend', 10),
 ('made_u_feel', 4),
 ('living_room_furniture', 4),
 ('not_recommend_staying', 4),
 ('not_big_deal', 4),
 ('great_place_stay', 4),
 ('let_u_know', 4),
 ('not_work_not', 3),
 ('give_4_star', 3)]

Now let's look at a word cloud of the N-grams in misleading reviews.

In [8]:
from wordcloud import WordCloud
from PIL import Image

ngrams_str = " ".join(n_grams)

# make house-shaped mask
mask = np.array(Image.open("../../house_shape.png"))
# fill transparent areas with white
mask[mask[:, :, 3] == 0] = [255, 255, 255, 255]

# generate wordcloud
wc = WordCloud(
    background_color="white", 
    max_words=1000, 
    width=mask.shape[0]*5,
    height=mask.shape[1]*5,
    mask = mask,
    contour_width=3,
    contour_color='steelblue',
    colormap="twilight_shifted"
).generate(ngrams_str)

wc

# show wordcloud
fig = px.imshow(wc, title=f"Most common {N}-grams in misleading reviews", width=700, height=700)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.show()

Here is a reusable function to add ngrams features to a dataframe.

In [9]:
def ngrams_features(n, prefix="ngrams_", corr_thresh=0.05):
    """Add n-gram features to the features dataframe."""

    # one-hot encode ngrams
    df["ngrams"] = df.comments.apply(lambda x: set(get_consecutive_ngrams(x, 3)))

    # get set of ngrams
    ng_set = set(series_to_ngrams(subset.comments, n).to_list())

    # one-hot encode ngrams
    ngram_features = {}
    for ngram in ng_set:
        ngram_features[prefix + ngram] = df.ngrams.apply(lambda ngrams: 1 if ngram in ngrams else 0)
    ngram_df = pd.DataFrame(ngram_features)
    
    # filter features on correlation with label
    ngrams_df = corr_filter(ngram_df, corr_thresh=corr_thresh)

    return ngrams_df

In [10]:
# ngrams_features(3, prefix="ngrams_", corr_thresh=0.09)

## "Mentioned Amenities" Features
Was an amenity that was included in the listing mentioned in the review?

First, we need to do some cleaning amenities to make them easier to match.

In [11]:
import re

def clean_amenities(amenities):
    """Clean the amenities column."""

    cleaned = []

    # basic cleaning
    for amenity in amenities:
        # remove quotes
        amenity = amenity.replace('"', "")
        # remove anything in parentheses or brackets
        amenity = re.sub(r"\(.*\)", "", amenity)
        amenity = re.sub(r"\[.*\]", "", amenity)
        # strip whitespace
        amenity = amenity.strip()
        # lowercase
        amenity = amenity.lower()
        # remove curly brackets
        amenity = amenity.replace("{", "").replace("}", "")


        cleaned.append(amenity)

    # split entries with a slash, "and", or "or"
    for to_split_on in ["/", " and ", " or "]:
        cleaned = [amenity.split(to_split_on) for amenity in cleaned]
        cleaned = [item.strip() for sublist in cleaned for item in sublist]

    # remove empty strings
    cleaned = [amenity for amenity in cleaned if amenity != ""]

    return cleaned

df.amenities = df.amenities.apply(clean_amenities)

Look at some examples of the cleaned amenities.

In [12]:
df.amenities.sample(1).values[0]

['tv',
 'cable tv',
 'wifi',
 'air conditioning',
 'pool',
 'kitchen',
 'free parking on premises',
 'pets allowed',
 'free street parking',
 'heating',
 'family',
 'kid friendly',
 'washer',
 'dryer',
 'smoke detector',
 'carbon monoxide detector',
 'essentials',
 'lock on bedroom door',
 'hangers',
 'hair dryer',
 'iron',
 'laptop friendly workspace',
 'self check-in',
 'lockbox',
 'bathtub',
 'hot water',
 'bed linens',
 'extra pillows',
 'blankets',
 'ethernet connection',
 'microwave',
 'coffee maker',
 'refrigerator',
 'dishwasher',
 'dishes',
 'silverware',
 'cooking basics',
 'oven',
 'stove',
 'single level home',
 'bbq grill',
 'patio',
 'balcony',
 'beach essentials',
 'long term stays allowed',
 'no stairs',
 'steps to enter',
 'well-lit path to entrance',
 'disabled parking spot',
 'no stairs',
 'steps to enter',
 'extra space around bed',
 'accessible-height bed',
 'no stairs',
 'steps to enter']

Calculate our amenities features, and look at the correlation between them and the label.

In [13]:
def amenities_features(corr_thresh=None, prefix="amenities_") -> pd.DataFrame:
    print(f"Calculating amenities features with correlation threshold of {corr_thresh}")
    amenities_features_df = pd.DataFrame()
    # 1. Find the amenities in the listings
    for index, row in df.iterrows():
        # 1. Find the amenities in the listings
        amenities = row.amenities

        # Add a new row to amenities_features_df
        amenities_features_df.loc[index, "id"] = index

        # 2. For each amenity, see if it is present in the review
        for amenity in amenities:
            if amenity in row.comments:
                # 3. If it is present, add 1 for that feature
                if amenity in amenities_features_df.columns:
                    amenities_features_df.loc[index, amenity] = 1
                #   If the amenity does not already exist from another review, add it to the features dataframe
                else:
                    amenities_features_df.loc[index, amenity] = 0
    
    # 4. fill missing values with 0
    amenities_features_df = amenities_features_df.fillna(0)

    # 6. Only keep amenities features that have a correlation with the label above a certain threshold
    amenities_features_df = corr_filter(amenities_features_df, corr_thresh=corr_thresh)

    # drop id column
    if "id" in amenities_features_df.columns:
        amenities_features_df = amenities_features_df.drop("id", axis=1)

    # prefix features
    amenities_features_df = amenities_features_df.add_prefix(prefix)

    return amenities_features_df


amen_features = amenities_features(corr_thresh=0.05, prefix="")
amen_features["review_id"] = df.review_id
amen_features["label"] = df.label
amen_features.head()

Calculating amenities features with correlation threshold of 0.05
Features above correlation threshold (0.05):
['id', 'lockbox', 'pool', 'dishwasher', 'tv', 'keypad', 'kitchen', 'breakfast']


,lockbox,pool,dishwasher,tv,keypad,kitchen,breakfast,review_id,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83097,good
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,133337,good
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,150928,good
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2706775,good
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,8602878,good


In [14]:
df.review_id.shape[0] == amen_features.review_id.shape[0]

True

Let's look at the amenities with the highest correlation to the label.

In [15]:
# OHE labels
labels_list: list = amen_features.label.unique().tolist()
dummies = amen_features.label.apply(lambda label: labels_list.index(label))

# calculate correlations with the label
corrs = amen_features.drop("review_id", axis=1).corrwith(dummies).sort_values(ascending=False)
corrs

C:\Users\grego\AppData\Local\Temp\ipykernel_19004\3136741035.py:6: FutureWarning:

The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



lockbox       0.090532
pool          0.070985
dishwasher    0.070433
tv            0.055847
keypad        0.055246
kitchen       0.050689
breakfast    -0.055234
dtype: float64

## Word Embeddings
Another way to look at the data is to use word embeddings. We can use the word2vec model from gensim to create word embeddings for each word in the dataset. We can then take the average of all word vectors in a review to get a vector representation of the review. We will then reduce the dimensionality of the vectors using PCA.

In [16]:
from gensim.models.word2vec import Word2Vec
from sklearn.decomposition import PCA
from typing import Tuple

def do_pca(features: pd.DataFrame, col: str, n_components: int) -> Tuple[np.ndarray, PCA]:
    """Perform PCA on embeddings."""
    print(f"Performing PCA on {col} with {n_components} components")
    pca = PCA(n_components=n_components, random_state=42)
    pca.fit(features)
    pca_embeddings = pca.transform(features)

    pca_features = pd.DataFrame()
    for i in range(pca_embeddings.shape[1]):
        pca_features[f"pca_{n_components}D_{col}_{i}"] = pca_embeddings[:, i]

    return pca_features, pca

def embeddings_features(emb_vector_size=300, col="comments") -> Tuple[pd.DataFrame, Word2Vec]:

    print(f"Embedding {col} with vector size {emb_vector_size}")
    w2v = Word2Vec(df[col].to_list(), 
            min_count=1,
            vector_size=emb_vector_size,
            workers=1,  # more workers make make the model non-deteministic, and we won't be able to save it to test on the test set
            window=3, 
            sg=1
        )
    
    # generate embeddings
    get_review_embeddings = lambda review: np.mean([w2v.wv[word] for word in review if word in w2v.wv], axis=0)
    embeddings = np.array([get_review_embeddings(review) for review in df[col].to_list()])
    
    # ignore DataFrame is highly fragmented.
    pd.options.mode.chained_assignment = None

    # add embeddings to features dataframe
    embd_features = pd.DataFrame()
    for i in range(embeddings.shape[1]):
        embd_features[f"embedding_{col}_{i}"] = embeddings[:, i]

    return embd_features, w2v

Let's try this out and visualize in 3D to see how well the reviews are separated by label.

In [17]:
comments_embeddings, _ = embeddings_features(col="comments")

pca_features = do_pca(comments_embeddings, "comments", 3)[0]

# add label from df.label
pca_features["label"] = df.label

pca_features

# use plotly to plot the PCA vectors
import plotly.express as px

fig = px.scatter_3d(pca_features, x="pca_3D_comments_0", y="pca_3D_comments_1", z="pca_3D_comments_2", color="label")

# make point smaller and opaque
fig.update_traces(marker=dict(size=2, opacity=0.5))

fig.show()

Embedding comments with vector size 300
Performing PCA on comments with 3 components


# Generate Features

With what we know, let's generate some features. I've provided a function for each type of feature, so we can mix and match.

In [18]:
to_try = {
    "pcas": [2, 3, 5, 10, 20],
    "embeddings": [50, 100, 200, 300, 500, 1000],
}

In [19]:
from dataclasses import dataclass
from typing import List, Any

@dataclass
class Feature:
    data: pd.DataFrame | np.ndarray
    col: str
    type: str
    models: dict | None
    params: dict | None

    def __str__(self):
        return f"{self.type} ({self.col}) - {self.params}"
    
    __repr__ = __str__
    

features: List[Feature] = []

# get the amenities
amenity005 = amenities_features(corr_thresh=0.05, prefix="amenity005_").fillna(0)
amenityall = amenities_features(corr_thresh=None, prefix="amenityall_").fillna(0)

features.extend([
    Feature(
        data=amenity005,
        col="amenities",
        type="amenities",
        models=None,
        params={
            "corr_thresh": 0.05, 
            "amenities": amenity005.columns.tolist()
        }
    ),
    Feature(
        data=amenityall,
        col="amenities",
        type="amenities",
        models=None,
        params={
            "corr_thresh": None, 
            "amenities": amenityall.columns.tolist()
            }
    ),
])

# get the embeddings
for emb_vector_size in to_try["embeddings"]:
    embeddings_comments, w2vmodel_comments = embeddings_features(col="comments", emb_vector_size=emb_vector_size)
    embeddings_description, w2vmodel_description = embeddings_features(col="description", emb_vector_size=emb_vector_size)
    
    features.extend([
        Feature(
            data=embeddings_comments, 
            col="comments", 
            type="embeddings", 
            models={
                "w2v": w2vmodel_comments
            }, 
            params={"vector_size": emb_vector_size}
        ),
        Feature(
            data=embeddings_description, 
            col="description", 
            type="embeddings", 
            models={
                "w2v": w2vmodel_description
            }, 
            params={"vector_size": emb_vector_size}
        ),
    ])

    
    for pca_dims in [3, 5, 10, 20]:

        # Apply PCA to the embeddings
        pca_features_comments, pca_comments = do_pca(embeddings_comments, "comments", pca_dims)
        pca_features_description, pca_description = do_pca(embeddings_description, "description", pca_dims)

        features.extend([
            Feature(
                data=pca_features_comments, 
                col="comments", type="pca", 
                models={
                    "pca": pca_comments,
                    "w2v": w2vmodel_comments
                }, 
                params={"n_components": pca_dims, "vector_size": emb_vector_size}
            ),
            Feature(
                data=pca_features_description, 
                col="description", 
                type="pca", 
                models={
                    "pca": pca_description,
                    "w2v": w2vmodel_description
                }, 
                params={"n_components": pca_dims, "vector_size": emb_vector_size}
            ),
        ])

        # Apply PCA to embeddings + amenities
        comments_and_amenities = pd.concat([embeddings_comments, amenityall], axis=1).fillna(0)
        description_and_amenities = pd.concat([embeddings_description, amenityall], axis=1).fillna(0)
        pca_features_comments_amenities, pca_comments_amenities = do_pca(comments_and_amenities, "comments+amenities", pca_dims)
        pca_features_description_amenities, pca_description_amenities = do_pca(description_and_amenities, "description+amenities", pca_dims)

        features.extend([
            Feature(
                data=pca_features_comments_amenities, 
                col="comments+amenities", 
                type="pca", 
                models={
                    "pca": pca_comments_amenities,
                    "w2v": w2vmodel_comments
                }, 
                params={"n_components": pca_dims, "vector_size": emb_vector_size}
            ),
            Feature(
                data=pca_features_description_amenities, 
                col="description+amenities", 
                type="pca", 
                models={
                    "pca": pca_description_amenities,
                    "w2v": w2vmodel_description
                }, 
                params={"n_components": pca_dims, "vector_size": emb_vector_size}
            ),
        ])

for feature in features:
    print(feature)


Calculating amenities features with correlation threshold of 0.05
Features above correlation threshold (0.05):
['id', 'lockbox', 'pool', 'dishwasher', 'tv', 'keypad', 'kitchen', 'breakfast']
Calculating amenities features with correlation threshold of None
Embedding comments with vector size 50
Embedding description with vector size 50
Performing PCA on comments with 3 components
Performing PCA on description with 3 components
Performing PCA on comments+amenities with 3 components
Performing PCA on description+amenities with 3 components
Performing PCA on comments with 5 components
Performing PCA on description with 5 components
Performing PCA on comments+amenities with 5 components
Performing PCA on description+amenities with 5 components
Performing PCA on comments with 10 components
Performing PCA on description with 10 components
Performing PCA on comments+amenities with 10 components
Performing PCA on description+amenities with 10 components
Performing PCA on comments with 20 compo

In [20]:
print(f"Total feature types: {len(features)}")

Total feature types: 110


## Save the features

Here we will save the features to a csv. We will also save the trained word2vec and pca models, so we can use it later.

In [21]:
import pickle

if isinstance(GEO, list):
    geo_display_name = "_".join(GEO)
else:
    geo_display_name = GEO

with open(f"../../data/features/features_{geo_display_name}.pkl", "wb") as f:
    pickle.dump(features, f)